In [2]:
!pip install pandas numpy transformers torch


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import sqlite3
from transformers import pipeline

c:\Users\amind\Desktop\projekt mupy\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def extract(csv_path):
    """
    Läser in rådata från CSV
    """
    df = pd.read_csv(csv_path)
    return df


In [5]:
def explore(df):
    print("Första rader:")
    display(df.head())

    print("\nInfo:")
    print(df.info())

    print("\nSaknade värden:")
    print(df.isna().sum())

In [ ]:
def clean_data(df):
    df = df.copy()

    df.columns = df.columns.str.lower().str.strip()

    if "order_date" in df.columns:
        df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")

    for col in ["price", "quantity", "delivery_days"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    df.fillna({
        "price": df["price"].median() if "price" in df.columns else 0,
        "quantity": 1,
        "delivery_days": df["delivery_days"].median() if "delivery_days" in df.columns else 0,
        "review_text": ""
    }, inplace=True)

    if "quantity" in df.columns:
        df = df[df["quantity"] > 0]

    return df


In [7]:
def add_features(df):
    df = df.copy()

    if "order_date" in df.columns:
        df["weekday"] = df["order_date"].dt.day_name()
        df["month"] = df["order_date"].dt.month
        df["week"] = df["order_date"].dt.isocalendar().week

    if {"price", "quantity"}.issubset(df.columns):
        df["total_sales"] = df["price"] * df["quantity"]

    return df

In [8]:
def add_sentiment(df):
    df = df.copy()

    if "review_text" not in df.columns:
        df["sentiment"] = "neutral"
        return df

    sentiment_model = pipeline(
        "sentiment-analysis",
        model="nlptown/bert-base-multilingual-uncased-sentiment"
    )

    def classify(text):
        if text.strip() == "":
            return "neutral"

        label = sentiment_model(text[:512])[0]["label"]
        score = int(label[0])

        if score <= 2:
            return "negative"
        elif score == 3:
            return "neutral"
        else:
            return "positive"

    df["sentiment"] = df["review_text"].apply(classify)
    return df


In [9]:
DATA_DICTIONARY = {
    "order_date": "Datum då ordern lades",
    "price": "Pris per enhet (rensad)",
    "quantity": "Antal enheter",
    "delivery_days": "Leveranstid i dagar",
    "review_text": "Kundens recension",
    "weekday": "Veckodag",
    "month": "Månad",
    "week": "Vecka",
    "total_sales": "Pris * antal",
    "sentiment": "Positiv / Neutral / Negativ (BERT)"
}

In [10]:
def load_to_sqlite(df, table_name, db_name="nordkaffe.db"):
    engine = create_engine(f"sqlite:///{db_name}")
    df.to_sql(table_name, engine, if_exists="replace", index=False)
    return engine

In [11]:
def verify(engine, table_name):
    count = pd.read_sql(f"SELECT COUNT(*) FROM {table_name}", engine)
    print("Antal rader i databasen:", count.iloc[0, 0])

In [12]:
def run_pipeline(csv_path, table_name):
    df = extract(csv_path)
    explore(df)

    df = clean_data(df)
    df = add_features(df)
    df = add_sentiment(df)

    engine = load_to_sqlite(df, table_name)
    verify(engine, table_name)

    return df
